# Data preparation and EDA



In [ ]:
!pip install emoji
!python -m spacy download ru_core_news_md
!pip install pymystem3
!pip install -q transformers datasets librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 20.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import string
import re
import emoji
import ru_core_news_md
from string import whitespace

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from collections import defaultdict

from pymystem3 import Mystem
from datasets import load_dataset

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
df = pd.read_csv('valid.csv')

In [ ]:
df.dtypes

,0
text,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
neutral,int64


In [ ]:
df.head()

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,1
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,1,0,0,1
2,Мне никогда в жизни не было так грустно!,0,0,0,0,1,0,0
3,Экономика находится под жестким контролем и су...,0,0,0,1,0,0,1
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,1,0,0,0


In [ ]:
df.shape

(5426, 8)

In [ ]:
df.nunique()

,0
text,5416
anger,2
disgust,2
fear,2
joy,2
sadness,2
surprise,2
neutral,2


In [ ]:
df['text'].value_counts()

,count
text,
Мне это нравится!,3
Спасибо!,3
СПАСИБО [ИМЯ],2
Большое спасибо.,2
Я взволнован!,2
...,...
"Я так рад, что вы добились справедливости и закрытия. Вы можете начать правильно лечиться уже сейчас. Я не уверен, что когда-нибудь получу то же самое, к сожалению",1
[ИМЯ] просто перестал получать мяч в 4-й четверти и встал в углу,1
"К сожалению, этот саб постепенно заселяется ревизионистами приквелов и старыми добрыми парнями.",1


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(5417, 8)

In [ ]:
tatget_columns = ['anger',	'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral']

# Functions for feature extruction

In [ ]:
def extract_hashtags(text: str) -> str:
  '''
  Get hashtags from text
  '''
  hashtags = re.findall(r'#\w+', text)
  return hashtags

In [ ]:
def extract_mentions(text: str) -> str:
  '''
  Get mentioned person from text
  '''
  mentions = re.findall(r'@\w+', text)
  return mentions

In [ ]:
def get_lenght(text: str) -> int:
  '''
  Get length of text
  '''
  return len(text)

In [ ]:
def count_exclamation(text: str) -> int:
  '''
  Count exclamation marks
  '''
  return text.count('!')

In [ ]:
def count_question(text: str) -> int:
  '''
  Count question marks
  '''
  return text.count('?')

In [ ]:
def count_positive_brackets(text: str) -> int:
  '''
  Calculate "positive" brackets
  '''
  return text.count(')')

In [ ]:
def count_negative_brackets(text: str) -> int:
  '''
  Calculate "negative" brackets
  '''
  return text.count('(')

In [ ]:
def get_emoji(text: str) -> str:
  '''
  Extract list of emojis
  '''
  res = []
  emojis = [emoji.emojize(word) for word in text.split()]
  for em in emojis:
    if emoji.is_emoji(em):
      res.append(em)
  return ' '.join(res)

In [ ]:
def get_num_emoji(emojies_list: list) -> int:
  '''
  Get number of emojies
  '''
  return len(emojies_list)

In [ ]:
def demojize(emojis_list: list) -> str:
  '''
  Convert emojis to text
  '''
  return ' '.join([emoji.demojize(em) for em in emojis_list])

In [ ]:
def get_url(text: str) -> str:
  '''
  Get urls from text.
  '''
  url = re.findall(r'https?://\S+', text)
  return url

In [ ]:
def get_word_count(text: str) -> int:
  '''
  Count number of words.
  '''
  return len(text.split())

In [ ]:
def get_avg_word_length(text: str) -> int:
  '''
  Count avarage lenght of words.
  '''
  words = text.split()
  word_count = len(words)
  avg_word_lengh = sum(len(word) for word in words) / word_count
  return avg_word_lengh

In [ ]:
def get_capitalized_words(text: str) -> str:
  '''
  Extract capitalazed words
  '''
  capitalazed_words = [word for word in text.split() if word.upper()]
  return ' '.join(capitalazed_words)

In [ ]:
def get_names(text: str) -> str:
  '''
  Extract named entities.
  '''
  nlp = ru_core_news_md.load()
  document = nlp(text)
  names = document.ents
  return ' '.join(names)

In [ ]:
#TODO Build pipeline
df['hashtags'] = df.text.map(extract_hashtags)
df['mentions'] = df.text.map(extract_mentions)
df['text_length'] = df.text.map(get_lenght)
df['num_exclamation'] = df.text.map(count_exclamation)
df['num_question'] = df.text.map(count_question)
df['num_positive_brackets'] = df.text.map(count_positive_brackets)
df['num_negative_brackets'] = df.text.map(count_negative_brackets)
df['emoji_list'] = df.text.map(get_emoji)
df['num_emojis'] = df.emoji_list.map(get_num_emoji)
df['emogies_as_word'] = df.emoji_list.map(demojize)
df['url'] = df.text.map(get_url)
df['num_words'] = df.text.map(get_word_count)
df['avg_word'] = df.text.map(get_avg_word_length)

# Clean text

In [ ]:
def handle_punctuation(inp_str: str) -> str:
  inp_str = str(inp_str)
  for punct in string.punctuation:
    inp_str = inp_str.replace(punct, ' ')
    inp_str = inp_str.replace('—', ' ')
    inp_str = inp_str.strip().lower()
  return inp_str


In [ ]:
cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

def remove_trash(text):
    allowed_chars = cyrillic_letters + whitespace
    return "".join([c for c in text if c in allowed_chars])

In [ ]:
# def lemma_text(text: str):
#   mystem = Mystem()
#   lemmas = mystem.lemmatize(text)
#   res = ''.join(lemmas)
#   return res

In [ ]:
def lemma_text(texts: str):
  mystem = Mystem()
  lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
  txtpart = lol(texts, 1000)
  res = []
  for txtp in txtpart:
    alltexts = ' '.join([txt + ' br ' for txt in txtp])

    words = mystem.lemmatize(alltexts)
    doc = []
    for txt in words:
        if txt != '\n' and txt.strip() != '':
            if txt == 'br':
                res.append(doc)
                doc = []
            else:
                doc.append(txt)
  return res

In [ ]:
def del_stopwords(text: str):
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('russian'))
  filtered_tokens = [word for word in tokens if word not in stop_words]
  return filtered_tokens

In [ ]:
df.text = df.text.map(handle_punctuation)
df.text = df.text.map(remove_trash)
df.text = df.text.map(del_stopwords)

In [ ]:
df.text = df.text.str.join(' ')

In [ ]:
df.text = lemma_text(df.text)

In [ ]:
df

,text,anger,disgust,fear,joy,sadness,surprise,neutral,hashtags,mentions,...,num_exclamation,num_question,num_positive_brackets,num_negative_brackets,emoji_list,num_emojis,emogies_as_word,url,num_words,avg_word
0,"[это, новый, орлеан, действительно, чувствоват...",0,0,0,0,0,0,1,[],[],...,0,2,0,0,,0,,[],11,5.363636
1,"[знать, ответ, чувак, запрограммировать, перех...",0,0,0,1,0,0,1,[],[],...,1,0,0,0,,0,,[],16,5.812500
2,"[жизнь, грустно]",0,0,0,0,1,0,0,[],[],...,1,0,0,0,,0,,[],8,4.125000
3,"[экономика, находиться, жесткий, контроль, суб...",0,0,0,1,0,0,1,[],[],...,0,0,0,0,,0,,[],20,6.250000
4,"[мочь, легко, взять, настоящий, камера, законн...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],18,5.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5421,"[это, довольно, опасный, государство, решать, ...",0,0,1,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,7.615385
5422,"[сегодня, утро, подавать, развод, надеяться, у...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],15,4.733333
5423,"[последний, это, случаться, просто, сказать, з...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,4.846154
5424,"[терпеть, мочь, высокомерный, придурок, форд, ...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],17,4.588235


In [ ]:
#df.to_csv('valid_lemma_df.csv')

In [ ]:
# from google.colab import files
# files.download('valid_lemma_df.csv')

In [ ]:
df

,text,anger,disgust,fear,joy,sadness,surprise,neutral,hashtags,mentions,...,num_exclamation,num_question,num_positive_brackets,num_negative_brackets,emoji_list,num_emojis,emogies_as_word,url,num_words,avg_word
0,"[это, новый, орлеан, действительно, чувствоват...",0,0,0,0,0,0,1,[],[],...,0,2,0,0,,0,,[],11,5.363636
1,"[знать, ответ, чувак, запрограммировать, перех...",0,0,0,1,0,0,1,[],[],...,1,0,0,0,,0,,[],16,5.812500
2,"[жизнь, грустно]",0,0,0,0,1,0,0,[],[],...,1,0,0,0,,0,,[],8,4.125000
3,"[экономика, находиться, жесткий, контроль, суб...",0,0,0,1,0,0,1,[],[],...,0,0,0,0,,0,,[],20,6.250000
4,"[мочь, легко, взять, настоящий, камера, законн...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],18,5.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5421,"[это, довольно, опасный, государство, решать, ...",0,0,1,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,7.615385
5422,"[сегодня, утро, подавать, развод, надеяться, у...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],15,4.733333
5423,"[последний, это, случаться, просто, сказать, з...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,4.846154
5424,"[терпеть, мочь, высокомерный, придурок, форд, ...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],17,4.588235


In [ ]:
df

,text,anger,disgust,fear,joy,sadness,surprise,neutral,hashtags,mentions,...,num_exclamation,num_question,num_positive_brackets,num_negative_brackets,emoji_list,num_emojis,emogies_as_word,url,num_words,avg_word
0,"[это, новый, орлеан, действительно, чувствоват...",0,0,0,0,0,0,1,[],[],...,0,2,0,0,,0,,[],11,5.363636
1,"[знать, ответ, чувак, запрограммировать, перех...",0,0,0,1,0,0,1,[],[],...,1,0,0,0,,0,,[],16,5.812500
2,"[жизнь, грустно]",0,0,0,0,1,0,0,[],[],...,1,0,0,0,,0,,[],8,4.125000
3,"[экономика, находиться, жесткий, контроль, суб...",0,0,0,1,0,0,1,[],[],...,0,0,0,0,,0,,[],20,6.250000
4,"[мочь, легко, взять, настоящий, камера, законн...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],18,5.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5421,"[это, довольно, опасный, государство, решать, ...",0,0,1,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,7.615385
5422,"[сегодня, утро, подавать, развод, надеяться, у...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],15,4.733333
5423,"[последний, это, случаться, просто, сказать, з...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],13,4.846154
5424,"[терпеть, мочь, высокомерный, придурок, форд, ...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],17,4.588235


In [ ]:
df.head(30)

,text,anger,disgust,fear,joy,sadness,surprise,neutral,hashtags,mentions,...,num_exclamation,num_question,num_positive_brackets,num_negative_brackets,emoji_list,num_emojis,emogies_as_word,url,num_words,avg_word
0,"[это, новый, орлеан, действительно, чувствоват...",0,0,0,0,0,0,1,[],[],...,0,2,0,0,,0,,[],11,5.363636
1,"[знать, ответ, чувак, запрограммировать, перех...",0,0,0,1,0,0,1,[],[],...,1,0,0,0,,0,,[],16,5.812500
2,"[жизнь, грустно]",0,0,0,0,1,0,0,[],[],...,1,0,0,0,,0,,[],8,4.125000
3,"[экономика, находиться, жесткий, контроль, суб...",0,0,0,1,0,0,1,[],[],...,0,0,0,0,,0,,[],20,6.250000
4,"[мочь, легко, взять, настоящий, камера, законн...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],18,5.611111
5,"[спасибо, доверие, статистика, мочь, достигать...",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],12,4.583333
6,"[вау, мама, другой, человек, называть, мой, ер...",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],16,4.687500
7,[оно],0,0,0,0,0,0,1,[],[],...,1,0,0,0,,0,,[],2,3.500000
8,"[крайний, мера, имя, время, завоевывать, доверие]",0,0,0,1,0,0,0,[],[],...,0,0,0,0,,0,,[],12,5.500000
9,"[хороший, хотеть, мир, отпрыск, трэш, либерал]",1,0,0,0,0,0,0,[],[],...,0,0,0,0,,0,,[],12,5.500000


In [ ]:
d = defaultdict(int)

texts = list(df.text)
for text in texts:
  for word in text:
    d[word] += 1



In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=False))

In [ ]:
d

{'запрограммировать': 1,
 'перехватывать': 1,
 'субсидироваться': 1,
 'распечатывать': 1,
 'отпрыск': 1,
 'трэш': 1,
 'разжигать': 1,
 'обезвоживание': 1,
 'рифмовать': 1,
 'мечтать': 1,
 'бродячий': 1,
 'ординатор': 1,
 'анестезиолог': 1,
 'выдувать': 1,
 'ускорение': 1,
 'дангит': 1,
 'озабоченный': 1,
 'сурова': 1,
 'кредитный': 1,
 'кавый': 1,
 'сиакама': 1,
 'овертайм': 1,
 'чемпионат': 1,
 'футбольный': 1,
 'мальк': 1,
 'информировать': 1,
 'переписываться': 1,
 'созревать': 1,
 'испечь': 1,
 'однодневный': 1,
 'альмонт': 1,
 'хранить': 1,
 'литр': 1,
 'фентанить': 1,
 'рубец': 1,
 'вдали': 1,
 'сбалансировать': 1,
 'смущенный': 1,
 'непреодолимый': 1,
 'антипрививка': 1,
 'фармвилль': 1,
 'франкфурт': 1,
 'взлет': 1,
 'биг': 1,
 'мак': 1,
 'олень': 1,
 'походить': 1,
 'измученный': 1,
 'заманивать': 1,
 'корона': 1,
 'королевство': 1,
 'дружить': 1,
 'терпение': 1,
 'отрывочный': 1,
 'врезаться': 1,
 'конфронтация': 1,
 'главное': 1,
 'восторжествовать': 1,
 'вероятный': 1,
 'ос

In [ ]:
del_list = [k for k, v in d.items() if v == 1]

In [ ]:
df.text

,text
0,"[это, новый, орлеан, действительно, чувствоват..."
1,"[знать, ответ, чувак, запрограммировать, перех..."
2,"[жизнь, грустно]"
3,"[экономика, находиться, жесткий, контроль, суб..."
4,"[мочь, легко, взять, настоящий, камера, законн..."
...,...
5421,"[это, довольно, опасный, государство, решать, ..."
5422,"[сегодня, утро, подавать, развод, надеяться, у..."
5423,"[последний, это, случаться, просто, сказать, з..."
5424,"[терпеть, мочь, высокомерный, придурок, форд, ..."


In [ ]:
def del_rare_words(text_lem, del_list=del_list):
  return [word for word in text_lem if word not in del_list]

In [ ]:
df.text = df.text.map(del_rare_words)

In [ ]:
df['len_lem'] = len(df.text)

In [ ]:
# df.to_csv('valid_lemma_df_1.csv')

# from google.colab import files
# files.download('valid_lemma_df_1.csv')

In [ ]:
d = defaultdict(int)

texts = list(df.text)
for text in texts:
  for word in text:
    d[word] += 1


In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=False))
d

{'орлеан': 2,
 'жесткий': 2,
 'нюанс': 2,
 'сегодняшний': 2,
 'сынок': 2,
 'завоевывать': 2,
 'фоновый': 2,
 'застревать': 2,
 'курение': 2,
 'нос': 2,
 'изначально': 2,
 'размножаться': 2,
 'награда': 2,
 'рулетка': 2,
 'близкие': 2,
 'химия': 2,
 'воскресный': 2,
 'находка': 2,
 'торговый': 2,
 'ошибочный': 2,
 'непопулярный': 2,
 'выпечка': 2,
 'невозмутимый': 2,
 'аптека': 2,
 'повторение': 2,
 'широкий': 2,
 'пан': 2,
 'детка': 2,
 'мрачный': 2,
 'светлый': 2,
 'новенький': 2,
 'крыса': 2,
 'дж': 2,
 'хрупкий': 2,
 'мафия': 2,
 'восхищаться': 2,
 'проверяться': 2,
 'уважать': 2,
 'неблокировать': 2,
 'знакомство': 2,
 'падать': 2,
 'геноцид': 2,
 'житель': 2,
 'дьявол': 2,
 'серебряный': 2,
 'крест': 2,
 'подход': 2,
 'худой': 2,
 'робот': 2,
 'болезненный': 2,
 'слева': 2,
 'брошюра': 2,
 'гулять': 2,
 'суббота': 2,
 'ранение': 2,
 'разозлиться': 2,
 'мигрень': 2,
 'безжалостный': 2,
 'размещение': 2,
 'ежедневный': 2,
 'молитва': 2,
 'таки': 2,
 'повышать': 2,
 'глотка': 2,
 'ул

In [ ]:
df.text = df.text.str.join(' ')

In [ ]:
df.to_csv('prep_valid.csv')

In [ ]:
from google.colab import files
files.download('prep_valid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>